# I. Выпуклость, необходимое условие минимума, двойственная задача

## 1. Критерий выпуклости второго порядка

Optimal allocation with resource constraint:
$$f_{i}\left(x_{i}\right) = a_{i}e^{x_{i}}, a_{i} > 0, i \in \{ 1, ..., n\}$$

Также ограничение бюджета:
$$\sum^{n}_{i=1}x_{i} = b, b \in R$$

Задача минимизации:
$$\sum^{n}_{i=1}a_{i}e^{x_{i}} \rightarrow min, a_{i} > 0, i \in \{ 1, ..., n\}$$

Итоговая задача:
$$\sum^{n}_{i=1}a_{i}e^{x_{i}} \rightarrow min, a_{i} > 0, i \in \{ 1, ..., n\}, \sum^{n}_{i=1}x_{i} = b, b \in R$$

Для изучения выпуклости рассмотрим гессиан оптимизируемой функции:
$$H(x)_{i,j} = 0, i \neq j$$
$$H(x)_{i,j} = a_{j}e^{x_{j}}, i = j$$
$$\text{При } a_{i} > 0 \text{ очевидно, что } H(x)_{i,i} = a_{i}e^{x_{i}} > 0, \text{ так как } \forall x \in R: e^{x_{i}} > 0$$
Матрица такого вида является положительно определенной.

Так же рассмотрим знакоопределенность гессиана ограничений:
$$H(x) = \textbf{0}$$ 

Таким образом, задача является выпуклой.

## 2. Условие оптимальности

Рассмотрим Лангранжиан системы:
$$L(x, \nu) = \sum^{n}_{i=1}a_{i}e^{x_{i}} - \nu (\textbf{1}^Tx-b)$$

Необходимое условие минимума: точки, где  производная лагранжиана обращается в 0, такие точки отметим звездой:
$$\nabla L (x^{\star}, \nu^{\star}) = \left( a_{1}e^{x_{1}^{\star}}, ... , a_{n}e^{x_{n}^{\star}} \right)^T - {\boldsymbol\nu^{\star}}^T  = 0$$

Выражая покомпонентно элементы вектора x*:
$$a_{i}e^{x_{i}^{\star}} = \nu^{\star}, i \in \{ 1, ..., n\}$$
$$x_{i}^{\star} = ln\left( \frac{\nu^{\star}}{a_{i}}\right), i \in \{ 1, ..., n\}$$

Таким образом условия оптимума ККТ будут иметь вид следующей системы:

\begin{cases}
  x_{i}^{\star} = ln\left( \frac{\nu^{\star}}{a_{i}}\right), i \in \{ 1, ..., n\}\\
  \textbf{1}^Tx^{\star} = b
\end{cases}

## 3. Двойственная задача

Двойственной к оригинальной задачей будет инфинум лагранжиана по вектору переменных x. Чтобы построить двойственную задачу, подставим в лагранжиан найденную выше точку оптимума.

$$g(\nu) = \underset{x}{inf} L(x, \nu)$$
$$g(\nu) = n \nu - \nu \left( \sum^{n}_{i=1}ln\left( \frac{\nu}{a_{i}}\right) -b \right) = \nu \left( n \left(1 - ln(\nu)\right) + \sum^{n}_{i=1}ln(a_i) + b \right)$$

Тогда двойственная задача есть:
$$\nu \left( n \left(1 - ln(\nu)\right) + \sum^{n}_{i=1}ln(a_i) + b \right) \rightarrow max$$

# II. Решение тестовых примеров

In [ ]:
import numpy as np
import cvxpy as cp
import time
import polars as pl
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 1. Генерация примеров

In [2]:
n_dims = range(10, 110, 10)
n_examples = 100
n_seeds = 100

In [3]:
raw_experiment_sheet = pl.DataFrame({"dim": n_dims}).select(
    pl.all().repeat_by(n_examples).flatten()
)

In [4]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    pl.col("dim")
    .map_elements(
        lambda dim_: np.random.uniform(low=0.1, high=10, size=dim_),
        return_dtype=pl.List(float),
        returns_scalar=False,
    )
    .alias("a"),
    pl.col("dim")
    .map_elements(
        lambda dim_: np.random.uniform(low=-10, high=10, size=1),
        return_dtype=float,
        returns_scalar=True,
    )
    .alias("b"),
)

In [5]:
raw_experiment_sheet

dim,a,b
i64,list[f64],f64
10,"[6.540121, 5.937077, … 6.738311]",-1.421913
10,"[9.194265, 5.101666, … 2.060314]",0.57337
10,"[4.387757, 1.16182, … 7.928853]",2.76018
10,"[9.102635, 1.87349, … 4.304656]",2.32229
10,"[6.857408, 9.455416, … 2.077583]",-5.056252
…,…,…
100,"[2.086696, 0.155142, … 3.867535]",0.318747
100,"[9.123822, 4.468367, … 6.809985]",9.127189
100,"[6.821365, 0.384294, … 7.545377]",-8.989801


## 2. Решение солвером CVXPY

In [6]:
def solve_oawrc(a, b):

    a = np.asarray(a)

    x = cp.Variable(a.shape[0])

    objective = cp.Minimize(cp.sum(a @ cp.exp(x)))

    constraints = [cp.sum(x) - b == 0]

    prob = cp.Problem(objective, constraints)

    try:
        start_t = time.time()
        result = prob.solve()
        sovle_time = time.time() - start_t
    except:
        res = {"cvxpy_x": None, "cvxpy_y": None, "cvxpy_t": None}
    else:
        res = {"cvxpy_x": list(x.value), "cvxpy_y": result, "cvxpy_t": sovle_time}

    return res

In [7]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    (
        pl.struct("a", "b")
        .map_elements(
            lambda x: solve_oawrc(**x),
            return_dtype=pl.Struct(
                {"cvxpy_x": pl.List(float), "cvxpy_y": float, "cvxpy_t": float}
            ),
        )
        .alias("cvxpy_out")
    )
).unnest("cvxpy_out")

In [8]:
raw_experiment_sheet

dim,a,b,cvxpy_x,cvxpy_y,cvxpy_t
i64,list[f64],f64,list[f64],f64,f64
10,"[6.540121, 5.937077, … 6.738311]",-1.421913,"[-0.434848, -0.338094, … -0.464706]",42.340207,0.024452
10,"[9.194265, 5.101666, … 2.060314]",0.57337,"[-0.723082, -0.134161, … 0.77275]",44.617356,0.014766
10,"[4.387757, 1.16182, … 7.928853]",2.76018,"[0.208135, 1.53701, … -0.383557]",54.030148,0.012615
10,"[9.102635, 1.87349, … 4.304656]",2.32229,"[-0.869806, 0.710937, … -0.121029]",38.140675,0.010477
10,"[6.857408, 9.455416, … 2.077583]",-5.056252,"[-0.981021, -1.302356, … 0.213104]",25.710213,0.009225
…,…,…,…,…,…
100,"[2.086696, 0.155142, … 3.867535]",0.318747,"[0.468086, 3.067032, … -0.149165]",333.230993,0.008897
100,"[9.123822, 4.468367, … 6.809985]",9.127189,"[-0.817832, -0.103955, … -0.525279]",402.728955,0.008288
100,"[6.821365, 0.384294, … 7.545377]",-8.989801,"[-0.585414, 2.290807, … -0.686408]",379.859047,0.009238


## 3. Решение прямой задачи методом Ньютона 

In [9]:
def random_sum_bound(dim, b):
    rnd = np.random.uniform(size=dim)
    return list(rnd / rnd.sum() * b)

In [10]:
raw_experiment_sheet = raw_experiment_sheet.select(
    pl.all().repeat_by(n_seeds).flatten()
).with_columns(
    pl.struct("dim", "b")
    .map_elements(lambda x: random_sum_bound(**x), return_dtype=pl.List(float))
    .alias("x_init")
)
raw_experiment_sheet = raw_experiment_sheet.with_columns(pl.lit(np.random.uniform(size=len(raw_experiment_sheet))).alias("nu_init"))

In [11]:
raw_experiment_sheet = raw_experiment_sheet.drop_nans()

In [12]:
def f(x, a):
    return np.dot(a, np.exp(x)).reshape(1,1)

def grad_f(x, a):
    return (a * np.exp(x)).reshape(1,-1)

def hessian_f(x, a):
    return np.diag(a * np.exp(x))

def g(x, b):
    return np.array([np.sum(np.asarray(x)) - b]).reshape(1,1)

def grad_g(x):
    return np.ones(len(x)).reshape(1,-1)

def hessian_g(x):
    return np.zeros([len(x)]*2)

In [13]:
def solve_oawrc_newton(a, b, x_init, nu_init, cvxpy_y, eps=0.01, max_iters=10000):
    sovle_time = 0
    newton_hist_y = []
    newton_hist_ops = []

    x = np.asarray(x_init)
    a = np.asarray(a)
    nu = nu_init
    
    for i in range(max_iters):
        iter_ops = 0
        start_t_iter = time.time()

        grad_L = grad_f(x, a) - grad_g(x) * nu
        hessian_L = hessian_f(x, a)

        iter_ops += int(3*grad_L.shape[-1])

        hessian_L_full = np.block([
            [hessian_L, grad_g(x).T],
            [grad_g(x), np.zeros([1,1])]
        ])
        
        grad_L_full = np.concatenate([grad_L, g(x, b).reshape(1,-1)], axis = -1)
        d = np.linalg.solve(hessian_L_full, -grad_L_full.T)

        iter_ops += int(hessian_L_full.shape[0]**3)

        x += d[:len(x),0]
        nu += d[len(x):,0]

        iter_ops += d.shape[0]

        sovle_time += time.time() - start_t_iter
        newton_hist_y.append(float(f(x, a)[0]))
        newton_hist_ops.append(int(i*iter_ops))

        if np.abs(f(x, a)[0] - cvxpy_y) < eps:
            break
    
    return {"newton_x": x.tolist(), "newton_y": f(x, a)[0], "newton_t": sovle_time, "newton_hist_ops": newton_hist_ops, "newton_hist_y": newton_hist_y}

In [14]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    (
        pl.struct("a", "b", "x_init", "nu_init", "cvxpy_y")
        .map_elements(
            lambda x: solve_oawrc_newton(**x),
            return_dtype=pl.Struct(
                {"newton_x": pl.List(float), "newton_y": float, "newton_t": float, "newton_hist_ops": pl.List(int), "newton_hist_y": pl.List(float)}
            ),
        )
        .alias("newton_out")
    )
).unnest("newton_out")

/tmp/ipykernel_57223/2671493511.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  newton_hist_y.append(float(f(x, a)[0]))


In [15]:
raw_experiment_sheet

dim,a,b,cvxpy_x,cvxpy_y,cvxpy_t,x_init,nu_init,newton_x,newton_y,newton_t,newton_hist_ops,newton_hist_y
i64,list[f64],f64,list[f64],f64,f64,list[f64],f64,list[f64],f64,f64,list[i64],list[f64]
10,"[6.540121, 5.937077, … 6.738311]",-1.421913,"[-0.434848, -0.338094, … -0.464706]",42.340207,0.024452,"[-0.250226, -0.154263, … -0.148234]",0.591354,"[-0.437117, -0.340378, … -0.466889]",42.341272,0.019287,"[0, 1372, 2744]","[44.335494, 42.466682, 42.341272]"
10,"[6.540121, 5.937077, … 6.738311]",-1.421913,"[-0.434848, -0.338094, … -0.464706]",42.340207,0.024452,"[-0.229003, -0.150289, … -0.043925]",0.676342,"[-0.438952, -0.342232, … -0.468654]",42.343553,0.000638,"[0, 1372, 2744]","[45.117556, 42.573098, 42.343553]"
10,"[6.540121, 5.937077, … 6.738311]",-1.421913,"[-0.434848, -0.338094, … -0.464706]",42.340207,0.024452,"[-0.100573, -0.114233, … -0.115444]",0.376444,"[-0.438862, -0.342215, … -0.468706]",42.34364,0.000673,"[0, 1372, 2744]","[45.168444, 42.576457, 42.34364]"
10,"[6.540121, 5.937077, … 6.738311]",-1.421913,"[-0.434848, -0.338094, … -0.464706]",42.340207,0.024452,"[-0.084767, -0.057075, … -0.166647]",0.406232,"[-0.437927, -0.341242, … -0.46782]",42.342309,0.000483,"[0, 1372, 2744]","[44.8344, 42.522079, 42.342309]"
10,"[6.540121, 5.937077, … 6.738311]",-1.421913,"[-0.434848, -0.338094, … -0.464706]",42.340207,0.024452,"[-0.042249, -0.232743, … -0.090528]",0.631019,"[-0.437406, -0.340871, … -0.467266]",42.34165,0.000808,"[0, 1372, 2744]","[44.500462, 42.488994, 42.34165]"
…,…,…,…,…,…,…,…,…,…,…,…,…
100,"[4.222479, 2.755177, … 7.286934]",-8.310487,"[-0.016524, 0.410461, … -0.562142]",415.347342,0.0076,"[-0.161772, -0.088202, … -0.000451]",0.436068,"[-0.017113, 0.409827, … -0.562774]",415.355437,0.001237,"[0, 1030702, … 5153510]","[683.473459, 492.583167, … 415.355437]"
100,"[4.222479, 2.755177, … 7.286934]",-8.310487,"[-0.016524, 0.410461, … -0.562142]",415.347342,0.0076,"[-0.072811, -0.0585, … -0.143472]",0.135594,"[-0.016706, 0.410235, … -0.562366]",415.348338,0.001229,"[0, 1030702, … 5153510]","[634.519614, 475.146837, … 415.348338]"
100,"[4.222479, 2.755177, … 7.286934]",-8.310487,"[-0.016524, 0.410461, … -0.562142]",415.347342,0.0076,"[-0.055949, -0.061476, … -0.145106]",0.496186,"[-0.016516, 0.410424, … -0.562177]",415.347373,0.001442,"[0, 1030702, … 6184212]","[703.530059, 499.547816, … 415.347373]"


## 4. Решение двойственной задачи методом Ньютона 

Для двойственной задачи шаг оптимизации метода Ньютона:
$$\nu^{k+1} = \nu^k - \left[\nabla^2 g(\nu^k)\right]^{-1} \nabla g(\nu^k)$$


В случае рассматриваемой двойственной функции $$g(\nu) = \nu \left( n \left(1 - ln(\nu)\right) + \sum^{n}_{i=1}ln(a_i) + b \right)$$

Градиент: 
$\nabla g(\nu) = - \nu ln(\nu) + \sum^{n}_{i=1}ln(a_i) + b $

Гессиан:
$\nabla^2 g(\nu) = - \frac{n}{\nu}$ 

Градиент и гессиан одномерные, поэтому шаг оптимизации упрощается: 
$$\nu^{k+1} = \nu^k - \frac{g'(\nu^k)}{g''(\nu^k)} $$

Число арифметических операций за одну итерацию - $2n + 6$; с учетом перевода $\nu$ в $x$ на каждой итерации для используемого критерия остановки $4n+6$.

In [16]:
def dual_g(nu, a, b):
    return nu * (len(a) * (1 - np.log(-nu)) + np.sum(np.log(a)) + b)

def grad_dual_g(nu, a, b):
    return - len(a) * np.log(nu) + np.sum(np.log(a)) + b

def hess_dual_g(nu, a):
    return - len(a) / nu

def nu_to_x(nu, a):
    return np.log(nu / a)

In [17]:
def solve_dual_newton(a, b, nu_init, cvxpy_y, eps=0.01, max_iters=10000):
    sovle_time = 0
    newton_hist_y = []

    a = np.asarray(a)
    nu = nu_init
    
    for i in range(max_iters):
        start_t_iter = time.time()

        grad = grad_dual_g(nu, a, b)
        hess = hess_dual_g(nu, a)

        nu -= grad / hess
        x = nu_to_x(nu, a)

        sovle_time += time.time() - start_t_iter
        newton_hist_y.append(float(f(x, a)[0]))

        if np.abs(f(x, a)[0] - cvxpy_y) < eps:
            break
    
    return {"dual_newton_x": x.tolist(), "dual_newton_y": f(x, a)[0], "dual_newton_t": sovle_time, "dual_newton_hist_y": newton_hist_y}

In [ ]:
raw_experiment_sheet = raw_experiment_sheet.with_columns(
    (
        pl.struct("a", "b", "nu_init", "cvxpy_y")
        .map_elements(
            lambda x: solve_dual_newton(**x),
            return_dtype=pl.Struct(
                {"dual_newton_x": pl.List(float), "dual_newton_y": float, "dual_newton_t": float, "dual_newton_hist_y": pl.List(float)}
            ),
        )
        .alias("dual_newton_out")
    )
).unnest("dual_newton_out")

# III. Анализ зависимостей

In [34]:
res_data = raw_experiment_sheet.clone()
res_data = (res_data
    .with_columns(pl.col("newton_hist_y").list.len().alias("iter_num"))
    .with_columns(pl.col("dual_newton_hist_y").list.len().alias("dual_iter_num"))
    .with_columns((pl.col("cvxpy_y") - pl.col("newton_y")).abs().alias("primal_diff"))
    .with_columns((pl.col("cvxpy_y") - pl.col("dual_newton_y")).abs().alias("dual_diff"))
    .select("dim", "newton_t", "dual_newton_t", "iter_num", "dual_iter_num", "primal_diff", "dual_diff")
    .group_by("dim")
    .agg(pl.all().mean().name.suffix("_mean"))
    .sort("dim")
)

In [39]:
fig = go.Figure()
fig2 = go.Figure()
fig3 = go.Figure()

fig.add_trace(go.Scatter(x=res_data["dim"], y=res_data["newton_t_mean"],mode="lines+markers", name="primal", line_shape='linear'))
fig.add_trace(go.Scatter(x=res_data["dim"], y=res_data["dual_newton_t_mean"],mode="lines+markers", name="dual", line_shape='linear'))

fig2.add_trace(go.Scatter(x=res_data["dim"], y=res_data["iter_num_mean"],mode="lines+markers", name="primal", line_shape='linear'))
fig2.add_trace(go.Scatter(x=res_data["dim"], y=res_data["dual_iter_num_mean"],mode="lines+markers", name="dual", line_shape='linear'))

fig3.add_trace(go.Scatter(x=res_data["dim"], y=res_data["primal_diff_mean"],mode="lines+markers", name="primal", line_shape='linear'))
fig3.add_trace(go.Scatter(x=res_data["dim"], y=res_data["dual_diff_mean"],mode="lines+markers", name="dual", line_shape='linear'))

fig.update_layout(
    title="Среднее время в зависимости от размерности переменной",
    xaxis_title="Размерность",
    yaxis_title="Среднее время",
    showlegend=True
)
fig2.update_layout(
    title="Среднее число итераций для достижения сходимости в зависимости от размерности переменной",
    xaxis_title="Размерность",
    yaxis_title="Среднее число итераций",
    showlegend=True
)
fig3.update_layout(
    title="Средний модуль разности значений ф-й в опт.точке с решением солвера в зависимости от размерности переменной",
    xaxis_title="Размерность",
    yaxis_title="Средний модуль разности значений ф-й в опт.точке",
    showlegend=True
)
fig.show()
fig2.show()
fig3.show()

In [28]:
def comp_plot(dim=10):

    init_comp_data = raw_experiment_sheet.filter(pl.col("dim") == dim).limit(6).to_pandas().reset_index()

    fig = make_subplots(rows=3, cols=2, vertical_spacing=0.1, horizontal_spacing=0.02)

    for row in range(1, 4):
        for col in range(1, 3):
            show_legend = False
            if row == 1 and col == 1:
                show_legend = True

            primal = init_comp_data.loc[row, "newton_hist_y"]
            dual = init_comp_data.loc[row, "dual_newton_hist_y"]
            s = init_comp_data.loc[row, "cvxpy_y"]

            fig.add_scatter(y=primal, row=row, col=col, name="primal", line=dict(color="blue"), showlegend=show_legend)
            fig.add_scatter(y=dual, row=row, col=col, name="dual", line=dict(color="red"), showlegend=show_legend)
            
            fig.add_scatter(x=[0, max(len(primal), len(dual)) - 1], y=[s, s], 
                            line=dict(dash="dash", color="green"), name="solver", showlegend=show_legend, row=row, col=col)
            

            fig.update_xaxes(title_text="Номер итерации", row=row, col=col, title_font=dict(size=10))
            fig.update_yaxes(title_text="f(x)", row=row, col=col, title_font=dict(size=10))

    fig.update_layout(
        title_text=f"Зависимость от начального решения при размерности {dim}",
        legend_title="Метод",
        width=1500,     
        height=800, 
        )
    fig.show()

comp_plot(dim=10)

In [29]:
comp_plot(dim=50)

# IV. Выводы

1. При увеличении размерности среднее время прямого метода растет быстрее по сравнению с двойственным из-за сложности вычисления матриц первых и вторых производных (время при dim=100 в 7 раз больше, чем при dim=10). Среднее время обратного метода также растет, но ощутимо медленее (менее, чем в 1.5 раза). 

2. Обратный метод сходится быстрее прямого, за исключением случая с размерностью 10. Число итераций до сходимости также увеличивается по мере увеличения размерности, но для обратного метода также значительно медленее. Для прямого метода с 3.5 до 11, для обратного с 4.7 до 5.1 с.

3. Если сравнивать среднюю разность значений функций в точке оптимума, найденных прямым\двойственныым методом и найденных с помощью солвера, то решение двойственного метода всегда точнее, причем точность увеличивается по мере увеличения размерности, а точность прямого метода, наоборот, растет.

4. Начальное решение несильно влияет на сходимость на маленькой размерности (10): в прямом методе значения функции на начальном решении очень близки, метод сходится за 2 итерации. В обратном методе значения функции в начальной точке отличаются, но метод сходится за 3-4 итерации.
На большей размерности (в примере 50) начальное решение влияет сильнее: в прямом методе значения целевой функции в начальной точке могут сильно различаться, в результате он сходится за 8-10 итераций, в обратном методе значения функции тоже отличаются (но в меньшем масштабе), что также влияет на сходимость (4-5 итераций).